In [1]:
import os

mingw_path = 'C:\\Users\\a1\\mingw\\mingw64\\bin'

os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
train = pd.read_csv('new_train_mean_cl.csv')
test = pd.read_csv('new_test_mean_cl.csv')

In [2]:
del test['Unnamed: 0']
del train['Unnamed: 0']

In [3]:
test = test.drop_duplicates(subset='id').set_index(keys='id').sort_index()

In [4]:
test = test[[u'Semana', u'Producto_ID', u'Cliente_ID', u'lag1', u'lag2', u'lag3', u'Agencia_ID', u'Canal_ID', u'Ruta_SAK',
             u'Cliente_ID_town_count', u'price', u'weight', u'pieces', u'cluster_nombre', u'drink', u'w_per_piece', u'OXXO', 
             u'ARTELI', u'ALSUPER', u'BODEGA', u'CALIMAX', u'XICANS', u'ABARROTES', u'CARNICERIA', u'FRUTERIA', 
             u'DISTRIBUIDORA', u'ELEVEN', u'HOTEL', u'HOSPITAL', u'CAFE', u'FARMACIA', u'CREME', u'SUPER', u'COMOD', 
             u'MODELOR', u'UNKN']]

In [5]:
def RMSLE_score(pred, true):
    score = np.power(pred-true, 2)
    return np.sqrt(np.mean(score))

In [6]:
from sklearn import cross_validation
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBRegressor
from sklearn import grid_search

In [7]:
X = train
y = train['Demanda_uni_equil_log0'].copy()
del train['Demanda_uni_equil_log0']

In [8]:
del X['drink']
del X['DISTRIBUIDORA']
del X['ARTELI']
del X['CALIMAX']
del X['MODELOR']
del X['HOSPITAL']
del X['HOTEL']
del test['drink']
del test['DISTRIBUIDORA']
del test['ARTELI']
del test['CALIMAX']
del test['MODELOR']
del test['HOSPITAL']
del test['HOTEL']

In [23]:
mean_submission = pd.read_csv('submit_mean.csv').set_index(keys='id').sort_index()
for w in [6, 7]: 
    train_index = (train['Semana'] == w)
    test_index = ~(train['Semana'] == w)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    xgbr = XGBRegressor(colsample_bytree=0.8,learning_rate=0.05, max_depth=15, n_estimators=100, reg_lambda=0.01, subsample=0.8)
    xgbr.fit(X_train, y_train)
    preds = xgbr.predict(X_test)
    preds[preds<0] = 0
    print RMSLE_score(preds, y_test)
    subms = xgbr.predict(test)
    mean_submission['xgb_demanda'+str(w)] = np.expm1(subms)
mean_submission.to_csv('subm_xgb_mean6.csv')

0.484640830711
0.47437522819


In [41]:
subms = xgbr.predict(test)

In [121]:
pd.Series(np.expm1(subms)).to_csv('subm_xgb.csv')

In [12]:
mean_submission = pd.read_csv('submit_mean.csv').set_index(keys='id').sort_index()

In [42]:
mean_submission['xgb_demanda'] = np.expm1(subms)

In [20]:
mean_submission['subm'] = 0.3*mean_submission['xgb_demanda6']+0.7*(0.3*mean_submission['xgb_demanda6']+0.3*mean_submission['xgb_demanda7']+
                                                                   0.2*mean_submission['xgb_demanda8']+0.2*mean_submission['xgb_demanda9'])

In [22]:
mean_submission['subm'].to_csv('subm_xgb.csv')

In [24]:
mean_submission

,Demanda_uni_equil,xgb_demanda6,xgb_demanda7
id,,,
0,3.77390,3.426123,5.721072
1,1.46508,1.458099,1.418618
2,2.08898,2.076922,2.007024
3,1.57699,1.437317,1.396789
4,3.18219,2.780858,3.154006
5,3.18412,2.787885,2.657208
6,1.29527,1.632302,1.403468
7,4.39736,4.314917,5.478069
8,4.83532,2.719286,2.701025


In [20]:
test.head()

,Semana,Producto_ID,Cliente_ID,lag1,lag2,lag3,Agencia_ID,Canal_ID,Ruta_SAK,Cliente_ID_town_count,...,ABARROTES,CARNICERIA,FRUTERIA,ELEVEN,CAFE,FARMACIA,CREME,SUPER,COMOD,UNKN
id,,,,,,,,,,,,,,,,,,,,,
0,11,35305,4639078,0.000000,1.609438,0.0,4037,1,2209,543723,...,0,0,0,0,0,0,0,0,0,1
1,11,1238,4705135,0.000000,0.000000,0.0,2237,1,1226,553984,...,0,0,0,0,0,0,0,0,0,1
2,10,32940,4549769,1.098612,1.098612,0.0,2045,1,2831,249200,...,0,0,0,0,0,0,0,0,0,1
3,11,43066,4717855,0.000000,0.000000,0.0,1227,1,4448,507639,...,0,0,0,0,0,0,0,0,0,1
4,11,1277,966351,0.000000,0.000000,0.0,1219,1,1130,572966,...,0,0,0,0,0,0,0,0,0,1
